In [1]:
import pandas as pd
import numpy as np

import pickle

from sklearn.feature_extraction.text import TfidfVectorizer 

from sklearn.svm import SVC 

import warnings
warnings.filterwarnings("ignore")

In [2]:
data_train = pd.read_csv('data_train_no_stemming.csv', sep=';')
data_test = pd.read_csv('data_test_no_stemming.csv', sep=';')

In [3]:
data_train.head()

,review_text,price,packaging,product,aroma
0,dair penasaran banget skincare reviewnya banya...,-1,0,0.0,0
1,ngincer bilang dupenya skii baca review tertaw...,0,0,1.0,0
2,suka tabirsurya teksturnya ringan menyerap ban...,0,1,1.0,0
3,nyobain pas muka lagi stress ny desperate bang...,0,0,-1.0,0
4,suka gokujyun losion merek hada labo lotionnya...,1,0,1.0,0


In [4]:
data_test.head()

,review_text,price,packaging,product,aroma
0,produk favorit banget enak dipake plus tidak r...,-1,0,1.0,0
1,lengket pas dipake hitungan tabirsurya yang ti...,1,0,0.0,0
2,minyak kontrol lumayan ngempesin jerawat efek ...,1,0,-1.0,0
3,favorite banget harganya terjangkau banget gak...,1,0,1.0,0
4,exfoliant toner pakai sebagi step double tonin...,0,0,1.0,0


In [5]:
data_train.isnull().sum()

review_text    0
price          0
packaging      0
product        0
aroma          0
dtype: int64

In [6]:
data_test.isnull().sum()

review_text    0
price          0
packaging      0
product        0
aroma          0
dtype: int64

In [7]:
X_train = data_train['review_text']
X_test = data_test['review_text']

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
fitur_train = TfidfVectorizer().fit(X_train)
tfidf_train = fitur_train.transform(X_train).toarray()
tfidf_test = fitur_train.transform(X_test).toarray()

In [9]:
def get_fitur_tfidf(tfidf, fitur):
    terms = tfidf.get_feature_names()
    sums = fitur.sum(axis = 0)

    data = []
    for col, term in enumerate(terms):
        data.append((term, sums[col] ))
    ranking = pd.DataFrame(data, columns=['Terms','TF-IDF'])
    return ranking

In [10]:
out_tfidf_tr = get_fitur_tfidf(fitur_train, tfidf_train)

In [11]:
out_tfidf_tr.shape

(9167, 2)

In [12]:
#FEATURE SELECTION CHI SQUARE
from sklearn import datasets
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [13]:
x_chi_tr = tfidf_train
x_chi_test = tfidf_test

In [14]:
#y_chi_tr = np.array(data_train[['price','packaging','aroma','product']])
#y_chi_test = np.array(data_test[['price','packaging','aroma','product']])

In [15]:
y_price_tr = (data_train['price'])
y_packaging_tr = (data_train['packaging'])
y_product_tr = (data_train['product'])
y_aroma_tr = (data_train['aroma'])

In [16]:
y_price_test = (data_test['price'])
y_packaging_test = (data_test['packaging'])
y_product_test = (data_test['product'])
y_aroma_test = (data_test['aroma'])

In [17]:
chi_price_tr = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_tr, y_price_tr)
chi_packaging_tr = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_tr, y_packaging_tr)
chi_product_tr = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_tr, y_product_tr)
chi_aroma_tr = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_tr, y_aroma_tr)


chi_price_test = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_test, y_price_test)
chi_packaging_test = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_test, y_packaging_test)
chi_product_test = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_test, y_product_test)
chi_aroma_test = SelectKBest(score_func=chi2, k=76).fit_transform(x_chi_test, y_aroma_test)

In [18]:
from sklearn.neighbors import KNeighborsClassifier

#aspek price
knn_price_7 = KNeighborsClassifier(n_neighbors=50)


knn_price_7.fit(chi_price_tr, y_price_tr)


predictions_price_7 = knn_price_7.predict(chi_price_test)

#aspek packaging

knn_packaging_7 = KNeighborsClassifier(n_neighbors=50)

knn_packaging_7.fit(chi_packaging_tr, y_packaging_tr)

predictions_packaging_7 = knn_packaging_7.predict(chi_packaging_test)

#aspek product

knn_product_7 = KNeighborsClassifier(n_neighbors=50)



knn_product_7.fit(chi_product_tr, y_product_tr)



predictions_product_7 = knn_product_7.predict(chi_product_test)

#aspek aroma

knn_aroma_7 = KNeighborsClassifier(n_neighbors=50)


knn_aroma_7.fit(chi_aroma_tr, y_aroma_tr)


predictions_aroma_7 = knn_aroma_7.predict(chi_aroma_test)


In [19]:
#Nilai price

knn7_price_skor=knn_price_7.score(chi_price_test,y_price_test)

#nilai packaging

knn7_packaging_skor=knn_packaging_7.score(chi_packaging_test,y_packaging_test)

#nilai product

knn7_product_skor=knn_product_7.score(chi_product_test,y_product_test)

#nilai aroma

knn7_aroma_skor=knn_aroma_7.score(chi_aroma_test,y_aroma_test)

print('=============== PRICE ===================')

print('Nilai Model price K-NN pada K-50           :',knn7_price_skor * 100 ,'%')

print('                                                                             ')
print('============= PACKAGING ================')

print('Nilai Model packaging K-NN pada K-50       :',knn7_packaging_skor * 100 ,'%')

print('                                                                             ')
print('============= PRODUCT ================')

print('Nilai Model product K-NN pada K-50         :',knn7_product_skor * 100 ,'%')

print('                                                                             ')
print('============= AROMA ================')

print('Nilai Model aroma K-NN pada K-50           :',knn7_aroma_skor * 100 ,'%')


=============== PRICE ===================
Nilai Model price K-NN pada K-50           : 53.15656565656566 %
                                                                             
============= PACKAGING ================
Nilai Model packaging K-NN pada K-50       : 84.72222222222221 %
                                                                             
============= PRODUCT ================
Nilai Model product K-NN pada K-50         : 67.8030303030303 %
                                                                             
============= AROMA ================
Nilai Model aroma K-NN pada K-50           : 80.3030303030303 %


In [20]:
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
#from sklearn.metrics import  multilabel_confusion_matrix

In [21]:
#Evaluasi aspek Price Pada K = 50
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_price_test,predictions_price_7)
print('Confusion Matrix\n')
print(confusion)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_price_test,predictions_price_7)))

print('Micro Precision: {:.2f}'.format(precision_score(y_price_test,predictions_price_7, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_price_test,predictions_price_7, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_price_test,predictions_price_7, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_price_test,predictions_price_7, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_price_test,predictions_price_7, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_price_test,predictions_price_7, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_price_test,predictions_price_7, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_price_test,predictions_price_7, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_price_test,predictions_price_7, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_price_test,predictions_price_7, target_names=['-1', '0', '1']))

Confusion Matrix

[[  4 129  19]
 [  1 405  13]
 [  0 209  12]]

Accuracy: 0.53

Micro Precision: 0.53
Micro Recall: 0.53
Micro F1-score: 0.53

Macro Precision: 0.54
Macro Recall: 0.35
Macro F1-score: 0.28

Weighted Precision: 0.52
Weighted Recall: 0.53
Weighted F1-score: 0.40

Classification Report

              precision    recall  f1-score   support

          -1       0.80      0.03      0.05       152
           0       0.55      0.97      0.70       419
           1       0.27      0.05      0.09       221

    accuracy                           0.53       792
   macro avg       0.54      0.35      0.28       792
weighted avg       0.52      0.53      0.40       792



In [22]:
#Evaluasi aspek Packaging pada K= 50
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_packaging_test,predictions_packaging_7)
print('Confusion Matrix\n')
print(confusion)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_packaging_test,predictions_packaging_7)))

print('Micro Precision: {:.2f}'.format(precision_score(y_packaging_test,predictions_packaging_7, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_packaging_test,predictions_packaging_7, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_packaging_test,predictions_packaging_7, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_packaging_test,predictions_packaging_7, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_packaging_test,predictions_packaging_7, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_packaging_test,predictions_packaging_7, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_packaging_test,predictions_packaging_7, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_packaging_test,predictions_packaging_7, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_packaging_test,predictions_packaging_7, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_packaging_test,predictions_packaging_7, target_names=['-1', '0', '1']))

Confusion Matrix

[[  0  37   4]
 [  0 670   2]
 [  0  78   1]]

Accuracy: 0.85

Micro Precision: 0.85
Micro Recall: 0.85
Micro F1-score: 0.85

Macro Precision: 0.33
Macro Recall: 0.34
Macro F1-score: 0.31

Weighted Precision: 0.74
Weighted Recall: 0.85
Weighted F1-score: 0.78

Classification Report

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        41
           0       0.85      1.00      0.92       672
           1       0.14      0.01      0.02        79

    accuracy                           0.85       792
   macro avg       0.33      0.34      0.31       792
weighted avg       0.74      0.85      0.78       792



In [23]:
#Evaluasi aspek Product pada K= 50
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_product_test,predictions_product_7)
print('Confusion Matrix\n')
print(confusion)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_product_test,predictions_product_7)))

print('Micro Precision: {:.2f}'.format(precision_score(y_product_test,predictions_product_7, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_product_test,predictions_product_7, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_product_test,predictions_product_7, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_product_test,predictions_product_7, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_product_test,predictions_product_7, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_product_test,predictions_product_7, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_product_test,predictions_product_7, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_product_test,predictions_product_7, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_product_test,predictions_product_7, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_product_test,predictions_product_7, target_names=['-1', '0', '1']))

Confusion Matrix

[[  1   0 134]
 [  1   0 120]
 [  0   0 536]]

Accuracy: 0.68

Micro Precision: 0.68
Micro Recall: 0.68
Micro F1-score: 0.68

Macro Precision: 0.39
Macro Recall: 0.34
Macro F1-score: 0.27

Weighted Precision: 0.54
Weighted Recall: 0.68
Weighted F1-score: 0.55

Classification Report

              precision    recall  f1-score   support

          -1       0.50      0.01      0.01       135
           0       0.00      0.00      0.00       121
           1       0.68      1.00      0.81       536

    accuracy                           0.68       792
   macro avg       0.39      0.34      0.27       792
weighted avg       0.54      0.68      0.55       792



In [24]:
#Evaluasi aspek Aroma Pada K=50
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_aroma_test,predictions_aroma_7)
print('Confusion Matrix\n')
print(confusion)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_aroma_test,predictions_aroma_7)))

print('Micro Precision: {:.2f}'.format(precision_score(y_aroma_test,predictions_aroma_7, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_aroma_test,predictions_aroma_7, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_aroma_test,predictions_aroma_7, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_aroma_test,predictions_aroma_7, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_aroma_test,predictions_aroma_7, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_aroma_test,predictions_aroma_7, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_aroma_test,predictions_aroma_7, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_aroma_test,predictions_aroma_7, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_aroma_test,predictions_aroma_7, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_aroma_test,predictions_aroma_7, target_names=['-1', '0', '1']))

Confusion Matrix

[[  1  33   6]
 [  0 607   8]
 [  0 109  28]]

Accuracy: 0.80

Micro Precision: 0.80
Micro Recall: 0.80
Micro F1-score: 0.80

Macro Precision: 0.83
Macro Recall: 0.41
Macro F1-score: 0.42

Weighted Precision: 0.80
Weighted Recall: 0.80
Weighted F1-score: 0.75

Classification Report

              precision    recall  f1-score   support

          -1       1.00      0.03      0.05        40
           0       0.81      0.99      0.89       615
           1       0.67      0.20      0.31       137

    accuracy                           0.80       792
   macro avg       0.83      0.41      0.42       792
weighted avg       0.80      0.80      0.75       792

